
# Quadratic Unconstrained Binary Optimization (QUBO) and Its Connection to the Ising Model

## What is a QUBO Problem?

A **Quadratic Unconstrained Binary Optimization (QUBO) problem** is a mathematical optimization problem defined as:

$$
\text{minimize: } x^T Q x
$$

- \$ x \$ is a vector of binary variables ($x_i \in \{0,1\}$)
- \$ Q \$ is a square matrix of real coefficients

The problem is called "unconstrained" because only the binary nature of variables is enforced—there are no other constraints. QUBO problems commonly appear in combinatorial optimization, including tasks like scheduling, portfolio optimization, and, in your case, RNA secondary structure prediction[^1][^2].

### QUBO Objective Function

$$
C(x) = \sum_{i} Q_{ii} x_i + \sum_{i < j} Q_{ij} x_i x_j
$$

The goal is to find the binary vector \$ x \$ that minimizes \$ C(x) \$.

## QUBO to Ising Model Mapping

Many quantum algorithms—especially those running on current quantum hardware—natively solve **Ising Hamiltonians**. The Ising model's variables are spin variables \$ s_i \in \{-1, +1\} \$:

$$
H = \sum_i h_i s_i + \sum_{i<j} J_{ij} s_i s_j
$$

### Mapping Steps

- Convert binary variables \$ x_i \in \{0,1\} \$ to spin variables \$ s_i \in \{-1, +1\} \$ with \$ x_i = (1 + s_i)/2 \$.
- Rewrite the QUBO cost function in terms of \$ s_i $. The resulting form will match the Ising Hamiltonian, with linear ($ h_i $) and pairwise ($ J_{ij} \$) terms.
- The problem of **minimizing the QUBO cost** is equivalent to **finding the ground state (lowest energy eigenstate) of the Ising Hamiltonian**.

Quantum algorithms like the **Variational Quantum Eigensolver (VQE)** or **Quantum Annealing** can then be used to find the ground state of the Ising Hamiltonian, effectively solving the original QUBO[^1].

## Example: QUBO to Ising Model in Python

This example demonstrates:

1. Definition of a simple QUBO problem
2. Mapping to the Ising model
3. Using Qiskit to check energies


In [2]:
from qiskit_optimization import QuadraticProgram
import numpy as np

# 1. Define your QUBO matrix
Q = np.array([
    [-2, 1],
    [ 1, -2]
])

# 2. Create a Qiskit Optimization problem
qp = QuadraticProgram()
qp.binary_var('x0')
qp.binary_var('x1')
qp.minimize(quadratic=Q, linear=[0, 0])

# 3. Convert to Ising Hamiltonian
qubit_op, offset = qp.to_ising()
print(f'Ising Hamiltonian: {qubit_op}')
print(f'Offset: {offset}')

# 4. Use scipy to find minimal solution directly (recommended for classical testing)
from qiskit.quantum_info import SparsePauliOp
from scipy.sparse.linalg import eigsh

H_mat = qubit_op.to_matrix(sparse=True)
eigvals, eigvecs = eigsh(H_mat, k=1, which='SA')
ground_energy = eigvals[0].real + offset
print("Minimum value (ground state energy):", ground_energy)

# DEMONSTRATE BITSTRING/SOLUTION: Map eigenvector back to variables
# The bitstring with largest amplitude in ground state is your optimal assignment
import numpy as np
max_idx = np.argmax(np.abs(eigvecs[:, 0]))
bit_fmt = f"0{qubit_op.num_qubits}b"
sol_bits = format(max_idx, bit_fmt)
opt_var_assignment = [int(x) for x in sol_bits[::-1]]
print("Optimal variable assignment:", opt_var_assignment)


Ising Hamiltonian: SparsePauliOp(['IZ', 'ZZ', 'ZI'],
              coeffs=[0.5+0.j, 0.5+0.j, 0.5+0.j])
Offset: -1.5
Minimum value (ground state energy): -2.0
Optimal variable assignment: [0, 1]


- Replace the Q matrix and number of variables for your specific problem.
- This code shows how to use Qiskit to represent the QUBO, convert it to an Ising Hamiltonian, and solve for the lowest energy—either classically or (with minimal change) on real quantum hardware.


## Summary Table: QUBO vs. Ising

| QUBO Variable | Ising Variable | Mapping Formula | Quantum Algorithm Target |
| :-- | :-- | :-- | :-- |
| \$ x_i \in \{0,1\} \$ | \$ s_i \in \{-1, +1\} \$ | \$ x_i = (1 + s_i)/2 \$ | Find Ising ground state energy |

## Conclusion

- **QUBO problems**: Widely applicable, combinatorial, and defined over binary variables.
- **Ising mapping**: Enables the use of quantum algorithms targeted at physical systems (qubits, spins).
- **Quantum solution**: Quantum computers simulate or find the ground state of the Ising Hamiltonian, which gives the QUBO minimum[^1][^2].

## VQE Implementation for Ground State Approximation ##
## Goal: ##
Use a parameterized quantum circuit (“ansatz”) to approximate the ground state (minimal free energy solution) of your system Hamiltonian that encodes RNA folding as an Ising model. The VQE algorithm will iteratively update quantum circuit parameters to minimize the expected energy.
- **Define the Problem Hamiltonian**
Suppose you’ve already mapped your QUBO to an Ising Hamiltonian (PauliSumOp or SparsePauliOp object in Qiskit):

In [4]:
from qiskit_optimization import QuadraticProgram
import numpy as np

Q = np.array([
    [-2, 1],
    [1, -2]
])
qp = QuadraticProgram()
qp.binary_var('x0')
qp.binary_var('x1')
qp.minimize(quadratic=Q, linear=[0, 0])
hamiltonian, offset = qp.to_ising()


- **2. Build a Parameterized Ansatz Circuit**
A simple example (2 qubit Ry + CNOT):

In [27]:
from qiskit.circuit.library import EfficientSU2

# Example: EfficientSU2 ansatz for 2 qubits
num_qubits = hamiltonian.num_qubits
ansatz = EfficientSU2(num_qubits, reps=1)

from qiskit_algorithms.optimizers import OptimizerResult
from qiskit_algorithms.optimizers import SPSA

optimizer = SPSA(maxiter=100)


/var/folders/1l/g3s1ytv57hv98hl39w041lxm0000gp/T/ipykernel_99900/523008681.py:5: DeprecationWarning: The class ``qiskit.circuit.library.n_local.efficient_su2.EfficientSU2`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.efficient_su2 instead.
  ansatz = EfficientSU2(num_qubits, reps=1)


ImportError: cannot import name 'BaseSampler' from 'qiskit.primitives' (/opt/anaconda3/lib/python3.12/site-packages/qiskit/primitives/__init__.py)

- **Run VQE Alog**


In [15]:
from qiskit.circuit.library import RealAmplitudes
ansatz = RealAmplitudes(num_qubits=hamiltonian.num_qubits, reps=2, entanglement='full')

from qiskit.primitives import StatevectorEstimator
from qiskit_algorithms import VQE
from qiskit.algorithms.optimizers import COBYLA

estimator = StatevectorEstimator()
vqe_solver = VQE(ansatz=ansatz, optimizer=COBYLA(), estimator=estimator)
result = vqe_solver.compute_minimum_eigenvalue(operator=qubit_op)
energy = result.eigenvalue.real + offset
print("Approximate ground state (minimal free energy):", energy)

/var/folders/1l/g3s1ytv57hv98hl39w041lxm0000gp/T/ipykernel_99900/4032174478.py:2: DeprecationWarning: The class ``qiskit.circuit.library.n_local.real_amplitudes.RealAmplitudes`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.real_amplitudes instead.
  ansatz = RealAmplitudes(num_qubits=hamiltonian.num_qubits, reps=2, entanglement='full')


ImportError: cannot import name 'Aer' from 'qiskit' (/opt/anaconda3/lib/python3.12/site-packages/qiskit/__init__.py)

In [1]:
import qiskit
print(qiskit.__version__)


2.1.1
